# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.114220e+02     1.807800e+00
 * time: 0.6047859191894531
     1     1.109280e+01     1.084703e+00
 * time: 2.662490129470825
     2    -1.120287e+01     1.150014e+00
 * time: 3.3061749935150146
     3    -3.351783e+01     8.523470e-01
 * time: 4.119194030761719
     4    -4.689529e+01     6.164556e-01
 * time: 4.932678937911987
     5    -5.674635e+01     2.184154e-01
 * time: 6.12213397026062
     6    -5.971593e+01     2.046794e-01
 * time: 6.6614830493927
     7    -6.089565e+01     8.122559e-02
 * time: 7.200860023498535
     8    -6.137248e+01     4.940483e-02
 * time: 7.7398130893707275
     9    -6.164080e+01     4.233046e-02
 * time: 8.510826110839844
    10    -6.181313e+01     3.005940e-02
 * time: 9.049865007400513
    11    -6.192095e+01     2.718549e-02
 * time: 9.593321084976196
    12    -6.200730e+01     1.794333e-02
 * time: 10.140278100967407
    13    -6.206993e+01     1.478953e-02
 * time: 10.679372072219849
    1

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671069
    AtomicLocal         -18.8557671
    AtomicNonlocal      14.8522637
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485373 
    Xc                  -19.3336821

    total               -62.261666460340
